# Problem

The attack success-rate of label #7 class has a gap between on-chip testing and simulation testing.

### e.g. from the statistics that I collectecd

|epoch|target label|patch size|MAX|lambda|success rate(on-chip)|success rate(simulation)|
|---- |----        |----     |----|----  |----                 |----                    |
|9|8|0.025|50|5e-4|68.29%|78.05%|
|9|7|0.025|50|5e-4|**13.89%**|**58.33%**|
|9|6|0.025|50|5e-4|44.74%|68.42%|
|9|5|0.025|50|5e-4|58.52%|64.71%|

We suspect that this is caused by the asynchronus convolution when running on-chip testing. So here we tried to inspect the difference of the numbers of spikes generated by the hidden layers.

In [1]:
from networks import GestureClassifierSmall
from sinabs.backend.dynapcnn import DynapcnnCompatibleNetwork
from sinabs.layers import SpikingLayer
from tqdm import tqdm
from aermanager.preprocess import create_raster_from_xytp

import os
import torch
import h5py
import numpy as np

finished bindings


### Define some global variables we need

In [2]:
OUTPUTS_OF_SPIKING_LAYERS = []
DEVICE = "cpu"

### Define the reset state function and hook function

In [3]:
def reset_states(net):
    for m in net.modules():
        if hasattr(m, "reset_states"):
            m.reset_states()


def hook_function(module, input_ts, output_ts):
    """forward hook function for nn.Modules that could save the sum of the outputs of hidden layers"""
    OUTPUTS_OF_SPIKING_LAYERS.append(output_ts.sum().item())

In [4]:
# load weights and convert to quantized snn
gesture_classifier = GestureClassifierSmall("BPTT_small_trained_200ms_2ms.pth", device=DEVICE)
gesture_classifier = gesture_classifier.to(DEVICE)
snn = gesture_classifier.model
quantized_model = DynapcnnCompatibleNetwork(
    snn,
    discretize=True,
    input_shape=(2, 128, 128),
)

In [5]:
# get spiking layers of the quantized model, we need to register forward hook functions on for them
spiking_layers = [each for each in quantized_model.sequence.modules() if isinstance(each, SpikingLayer)]

# register forward hook functions
for lyr in spiking_layers:
    lyr.register_forward_hook(hook_function)

In [7]:
# loading the input data from h5 files
# by changing the TARGET_LABEL and N_EPOCH, you can read different attack patches.
N_EPOCH = 9
TARGET_LABEL = 8
MAX = 50
PATCH_SIZE = 0.025

attack_file_name = f"attacks_patches_ep{N_EPOCH}_lb{TARGET_LABEL}_num{MAX}_patchsize{str(PATCH_SIZE).split('.')[1]}.h5"
attack_file = os.path.join("./attack_patches", attack_file_name)
# read data from file
data = h5py.File(attack_file, "r")
successful_attacks = np.where(data["attack_successful"])[0]

In [8]:
# start to inspect the activation of the hidden layers
for idx in successful_attacks:
    print("=" * 50)
    origin_spk = data["original_spiketrains"][str(idx)]
    attacked_spk = data["attacked_spiketrains"][str(idx)]
    ground_truth = data["ground_truth"][idx]
    assert ground_truth == data["sinabs_label"][idx]
    print(f"Target: {TARGET_LABEL}, ground truth: {ground_truth}")
    
    reset_states(quantized_model.sequence)
    raster_attacked = create_raster_from_xytp(attacked_spk, dt=500, bins_x=np.arange(129), bins_y=np.arange(129))
    output_ts = quantized_model(torch.tensor(raster_attacked).to(DEVICE)).squeeze().sum(0)
    print(f"N. spikes outputs by last layer: {output_ts.sum()}, activations for attacked output: {output_ts.detach().numpy().astype(int)}")
    print(f"N. spikes outputs by hidden layer from attacked input: {OUTPUTS_OF_SPIKING_LAYERS}")
    OUTPUTS_OF_SPIKING_LAYERS.clear()

Target: 8, ground truth: 7
N. spikes outputs by last layer: 97.0, activations for attacked output: [12  0  4  8  0  8 15  4 22 15  9]
N. spikes outputs by hidden layer from attacked input: [45924.0, 26803.0, 2934.0, 97.0]
Target: 8, ground truth: 7
N. spikes outputs by last layer: 69.0, activations for attacked output: [15  0  0  0  0  2  5 18 19  5  5]
N. spikes outputs by hidden layer from attacked input: [66986.0, 30213.0, 3674.0, 69.0]
Target: 8, ground truth: 5
N. spikes outputs by last layer: 101.0, activations for attacked output: [24  0 11  0  1  7 12  1 26  7 12]
N. spikes outputs by hidden layer from attacked input: [64344.0, 36406.0, 3941.0, 101.0]
Target: 8, ground truth: 3
N. spikes outputs by last layer: 147.0, activations for attacked output: [22  1  1 23 16 10 10  3 32 13 16]
N. spikes outputs by hidden layer from attacked input: [54193.0, 36134.0, 3243.0, 147.0]
Target: 8, ground truth: 0
N. spikes outputs by last layer: 132.0, activations for attacked output: [19  0  